# Ejercicio 16
10 categorías que tienen la menor cantidad de contenido anónimo publicado.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [5]:
df_contents = sqlContext.read.parquet('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/contents.parquet', header=True, inferSchema=True)
df_categorylinks = sqlContext.read.parquet('/content/drive/MyDrive/2021-2C/ORGANIZACION DE DATOS (7506)/TPs/TP2/Wikipedia_dataset-2c2021/categorylinks.parquet', header=True, inferSchema=True)

In [6]:
df_contents

DataFrame[title: string, id: bigint, namespace: bigint, revision_id: bigint, parent_revision: double, revision_timestamp: string, revisor_username: string, revisor_id: double, revisor_ip: string, revisor_comment: string]

In [7]:
df_categorylinks

DataFrame[cl_from: bigint, cl_to: string, cl_sortkey: string, cl_timestamp: string, cl_sortkey_prefix: string, cl_collation: string, cl_type: string]

In [8]:
rdd_contents = df_contents.rdd
rdd_categorylinks = df_categorylinks.rdd

In [9]:
rdd_contents.take(5)

[Row(title='Wikipedia:Artículos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor_username='Lojwe

In [10]:
rdd_categorylinks.take(5)

[Row(cl_from=5, cl_to='Wikipedia:Mantenimiento', cl_sortkey='*LP:.R@FN\x04NF@:.:P*0FN\x03\x06*LP:.R@FN\x04NF@:.:P*0FN\x01,\x88\x15\x01Ü±Ü\x19', cl_timestamp='2018-06-06 21:42:29', cl_sortkey_prefix='Articulos solicitados', cl_collation='uca-es-u-kn', cl_type='page'),
 Row(cl_from=5, cl_to='Wikipedia:PÃ¡ginas_con_enlaces_mÃ¡gicos_de_ISBN', cl_sortkey='*LP:.R@FN\x04NF@:.:P*0FN\x01B\x88\x15\x01Ü\x19', cl_timestamp='2021-04-01 03:18:11', cl_sortkey_prefix=None, cl_collation='uca-es-u-kn', cl_type='page'),
 Row(cl_from=7, cl_to='Andorra', cl_sortkey='\x04\x03\x06*D0FLL*\x01\n\x01ÄÜ\n', cl_timestamp='2013-01-12 19:07:41', cl_sortkey_prefix=' ', cl_collation='uca-es-u-kn', cl_type='page'),
 Row(cl_from=7, cl_to='Instituciones_patrocinadoras_de_equipos_ciclistas', cl_sortkey='*D0FLL*\x01\x0b\x01Ü\n', cl_timestamp='2011-03-30 19:52:38', cl_sortkey_prefix=None, cl_collation='uca-es-u-kn', cl_type='page'),
 Row(cl_from=7, cl_to='Wikipedia:ArtÃ\xadculos_con_datos_locales', cl_sortkey='*D0FLL*\x01\

In [11]:
rdd_ip_content = rdd_contents.filter(lambda x: x.revisor_ip != None)

In [12]:
rdd_ip_content.count()

193286

In [13]:
contentsByIp = rdd_ip_content.map(lambda x: (x.id,1))

In [14]:
contentsByIp.take(5)

[(7, 1), (46, 1), (77, 1), (81, 1), (160, 1)]

In [15]:
categoryByIp = rdd_categorylinks.map(lambda x: (x.cl_from,x.cl_to))

In [16]:
categoryByIp.take(5)

[(5, 'Wikipedia:Mantenimiento'),
 (5, 'Wikipedia:PÃ¡ginas_con_enlaces_mÃ¡gicos_de_ISBN'),
 (7, 'Andorra'),
 (7, 'Instituciones_patrocinadoras_de_equipos_ciclistas'),
 (7, 'Wikipedia:ArtÃ\xadculos_con_datos_locales')]

In [17]:
categoryWithIp = contentsByIp.join(categoryByIp)

In [18]:
categoryWithIp.take(5)

[(5490, (1, 'Iglesia_catÃ³lica_en_EspaÃ±a')),
 (5490, (1, 'Semana_Santa_en_EspaÃ±a')),
 (5490, (1, 'Wikipedia:ArtÃ\xadculos_con_pasajes_que_requieren_referencias')),
 (6366, (1, 'Juegos_infantiles')),
 (6366, (1, 'Juegos_tradicionales'))]

In [19]:
least_anonymous_categories  = categoryWithIp.map(lambda x: (x[1][1],1)).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x: x[1])

In [20]:
least_anonymous_categories

[('Equipos_de_FÃ³rmula_3', 1),
 ('RÃ\xados_de_la_vertiente_Ã\xadndica_de_Mozambique', 1),
 ('Heteras', 1),
 ('Historia_de_Palos_de_la_Frontera', 1),
 ('Nacidos_en_Palos_de_la_Frontera', 1),
 ('Obras_de_G._K._Chesterton', 1),
 ('PelÃ\xadculas_en_isixhosa', 1),
 ('Brahmaputra', 1),
 ('Organizaciones_no_gubernamentales_de_Colombia', 1),
 ('Argentina_en_1815', 1)]

# Respuesta

In [21]:
print("Las 10 categorías que tienen la menor cantidad de contenido anónimo publicado son")

for c in least_anonymous_categories:
  print("\t",c[0])

Las 10 categorías que tienen la menor cantidad de contenido anónimo publicado son
	 Equipos_de_FÃ³rmula_3
	 RÃ­os_de_la_vertiente_Ã­ndica_de_Mozambique
	 Heteras
	 Historia_de_Palos_de_la_Frontera
	 Nacidos_en_Palos_de_la_Frontera
	 Obras_de_G._K._Chesterton
	 PelÃ­culas_en_isixhosa
	 Brahmaputra
	 Organizaciones_no_gubernamentales_de_Colombia
	 Argentina_en_1815
